Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing.png)

# Automated Machine Learning
**_Bank Marketingデータセットを使用した分類モデルの作成とデプロイメント_**

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Train](#Train)
1. [Results](#Results)
1. [Deploy](#Deploy)
1. [Test](#Test)
1. [Acknowledgements](#Acknowledgements)

本NotebookはGitHubで公開されているサンプルを日本語化し、一部の手順を変更及び簡略化したものになります。サンプルの最新バージョンについては以下のURLを参照ください。  
https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing-all-features.ipynb

## Introduction

このサンプルでは、UCI Bank Marketingデータセットを使用して、Auto MLを使った分類モデルの作成とAzure Container Instance(ACI)へのモデルのデプロイを行います。分類モデルのゴールは、 依頼者が銀行に定期預金を申し込むかどうかを予測できるようにすることです。

また、こちらの手順では作成済みのAzure Machine Learningワークスペース及びコンピューティングインスタンスを使用することを前提としています。作成済みでない場合には、Notebook [configuration](../../../configuration.ipynb)を実行してAzure MLワークスペースを作成し、接続できることを確認してください。  

このNotebookでは、以下のことを学習します：
1. ワークスペースを使い、実験を作成
2. `AutoMLConfig`を使い、Auto MLを構成
3. コンピューティングインスタンスを使用して、ONNX互換のモデルを訓練
4. 訓練結果及び特徴づけ手順を確認し、ONNXモデルを保存
5. ONNXモデルによる推論の実施
6. モデルの登録
7. コンテナイメージの作成
8. Azure Container Instance(ACI)サービスの作成
9. ACIサービスのテスト

ONNXについては、関連ドキュメント [here](https://github.com/onnx/onnx)を参照してください。

## Setup

Azure Machine Learningワークスペースが作成済みであるとします。Auto MLを使うためには、`ワークスペース`の中に`実験`オブジェクトを作成します。

In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.explain.model._internal.explanation_client import ExplanationClient

ワークスペースにアクセスするためには、Azureの認証が必要になります。

デフォルトの認証は、デフォルトのテナントに対して対話的に認証を行います。下のセル内で実行している`ws = Workspace.from_config()`ラインで、最初の実行時に認証を行います。

In [ ]:
#ワークスペースにアクセスするための認証
ws = Workspace.from_config()

# 実験名を指定
experiment_name = 'automl-classification-bmarketing-all'

# 実験オブジェクトを生成
experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

# Data

### Load Data

Tabularデータセットとして、Bank Marketingデータセットをロードします。

### Training Data

In [ ]:
# 訓練データのロード
data = pd.read_csv("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
data.head()

In [ ]:
# 欠損データを追加
import numpy as np

missing_rate = 0.75
n_missing_samples = int(np.floor(data.shape[0] * missing_rate))
missing_samples = np.hstack((np.zeros(data.shape[0] - n_missing_samples, dtype=np.bool), np.ones(n_missing_samples, dtype=np.bool)))
rng = np.random.RandomState(0)
rng.shuffle(missing_samples)
missing_features = rng.randint(0, data.shape[1], n_missing_samples)
data.values[np.where(missing_samples)[0], missing_features] = np.nan

In [ ]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# 訓練データを、CSV形式でデータストアに保存
pd.DataFrame(data).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)

# 訓練データを、Tabularデータセットとして再ロード
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

### Validation Data

In [ ]:
# 評価データセットをロードし、Tabularデータセットに変換
validation_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_validate.csv"
validation_dataset = Dataset.Tabular.from_delimited_files(validation_data)

### Test Data

In [ ]:
# テストデータセットをロードし、Tabularデータセットに変換
test_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv"
test_dataset = Dataset.Tabular.from_delimited_files(test_data)

## Train

AutoMLConfigオブジェクトをインスタンス化します。これは、実験の実行時に使用されるセットとデータになります。

|Property|Description|
|-|-|
|**task**|classification or regression or forecasting|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**blacklist_models** | *List* of *strings* indicating machine learning algorithms for AutoML to avoid in this run. <br><br> Allowed values for **Classification**<br><i>LogisticRegression</i><br><i>SGD</i><br><i>MultinomialNaiveBayes</i><br><i>BernoulliNaiveBayes</i><br><i>SVM</i><br><i>LinearSVM</i><br><i>KNN</i><br><i>DecisionTree</i><br><i>RandomForest</i><br><i>ExtremeRandomTrees</i><br><i>LightGBM</i><br><i>GradientBoosting</i><br><i>TensorFlowDNN</i><br><i>TensorFlowLinearClassifier</i><br><br>Allowed values for **Regression**<br><i>ElasticNet</i><br><i>GradientBoosting</i><br><i>DecisionTree</i><br><i>KNN</i><br><i>LassoLars</i><br><i>SGD</i><br><i>RandomForest</i><br><i>ExtremeRandomTrees</i><br><i>LightGBM</i><br><i>TensorFlowLinearRegressor</i><br><i>TensorFlowDNN</i><br><br>Allowed values for **Forecasting**<br><i>ElasticNet</i><br><i>GradientBoosting</i><br><i>DecisionTree</i><br><i>KNN</i><br><i>LassoLars</i><br><i>SGD</i><br><i>RandomForest</i><br><i>ExtremeRandomTrees</i><br><i>LightGBM</i><br><i>TensorFlowLinearRegressor</i><br><i>TensorFlowDNN</i><br><i>Arima</i><br><i>Prophet</i>|
| **whitelist_models** |  *List* of *strings* indicating machine learning algorithms for AutoML to use in this run. Same values listed above for **blacklist_models** allowed for **whitelist_models**.|
|**experiment_exit_score**| Value indicating the target for *primary_metric*. <br>Once the target is surpassed the run terminates.|
|**experiment_timeout_hours**| Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**| Flag to enble early termination if the score is not improving in the short term.|
|**featurization**| 'auto' / 'off'  Indicator for whether featurization step should be done automatically or not. Note: If the input data is sparse, featurization cannot be turned on.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

**_主要メトリックについての詳細は、 [here](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)を参照ください_**

In [ ]:
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "iterations": 10,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             experiment_exit_score = 0.9984,
                             blacklist_models = ['KNN','LinearSVM'],
                             enable_onnx_compatible_models=True,
                             training_data = train_data,
                             label_column_name = label,
                             validation_data = validation_dataset,
                             enable_local_managed=True,
                             **automl_settings
                            )

実験オブジェクトに対して`submit`メソッドを実行し、分類モデルを作成します。

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# 実行結果を表示
local_run

実行結果に対して、最適モデルを取得

In [ ]:
best_run, fitted_model = local_run.get_output()

## Transparency

特徴づけ手順のサマリを表示します

In [ ]:
featurizer = fitted_model.named_steps['datatransformer']
df = featurizer.get_featurization_summary()
pd.DataFrame(data=df)

`is_user_friendly=False`を指定することで詳細を表示します

In [ ]:
df = featurizer.get_featurization_summary(is_user_friendly=False)
pd.DataFrame(data=df)

## Results

In [ ]:
from azureml.widgets import RunDetails

# 実行結果の表示
RunDetails(local_run).show()

### Retrieve the Best ONNX Model

最適な実行結果とモデルを、`get_output`メソッドで取得します。モデルはパイプラインと前処理情報を含んでいます。また、`get_output`メソッドでは、任意のメトリックや実行結果に対して、モデルを抽出できます。

`return_onnx_model_True`パラメータを設定することで、Pythonモデルの代わりにONNXモデルを抽出できます。

In [ ]:
best_run, onnx_mdl = local_run.get_output(return_onnx_model=True)

### Save the best ONNX model

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

# ONNXモデルの保存
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

### Predict with the ONNX model, using onnxruntime package

In [ ]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    test_df = test_dataset.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(test_df)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

## Deploy

### Retrieve the Best Model

最適な実行結果とモデルを、`get_output`メソッドで取得します。モデルはパイプラインと前処理情報を含んでいます。また、`get_output`メソッドでは、任意のメトリックや実行結果に対して、モデルを抽出できます。

### Register the Fitted Model for Deployment
`register_model`メソッドの呼び出しにおいて`metric`や`iteration`を指定しない場合は、主要メトリックの値が最も良いモデルが登録されます

In [ ]:
# 最適なモデルを登録
model_name = best_run.properties['model_name']
description = 'AutoML Model trained on bank marketing data to predict if a client will subscribe to a term deposit'
tags = None

model = local_run.register_model(model_name = model_name, description = description, tags = tags)

print(local_run.model_id) # This will be written to the script file later in the notebook.

### Deploy the model as a Web Service on Azure Container Instance

In [ ]:
best_run, fitted_model = local_run.get_output()
model_name = best_run.properties['model_name']
print(model_name)

script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/env.yml')

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'

myenv = Environment.from_conda_specification(name="myenv", file_path=conda_env_file_name)
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "bmData", 'type': "automl_classification"}, 
                                               description = 'sample service for Automl Classification')

aci_service_name = 'automl-sample-bankmarketing'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Test

最適なモデルが作成されたので、テストデータを使って予測を行います

In [ ]:
from numpy import array

X_test = test_dataset.drop_columns(columns=['y'])
y_test = test_dataset.keep_columns(columns=['y'], validate=True)
test_dataset.take(5).to_pandas_dataframe()

In [ ]:
X_test = X_test.to_pandas_dataframe()
y_test = y_test.to_pandas_dataframe()

y_pred  = fitted_model.predict(X_test)

予測結果に対する混同行列を表示します

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(y_test, y_pred)
print(conf_mx)

ACIサービスを呼び出して、予測を行います

In [ ]:
test = json.dumps({"data": X_test.values.tolist()})
y_pred2 = aci_service.run(test)

print(y_pred2)

## Acknowledgements

This Bank Marketing dataset is made available under the Creative Commons (CCO: Public Domain) License: https://creativecommons.org/publicdomain/zero/1.0/. Any rights in individual contents of the database are licensed under the Database Contents License: https://creativecommons.org/publicdomain/zero/1.0/ and is available at: https://www.kaggle.com/janiobachmann/bank-marketing-dataset .

_**Acknowledgements**_
This data set is originally available within the UCI Machine Learning Database: https://archive.ics.uci.edu/ml/datasets/bank+marketing

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014